In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive/') 

!mkdir "/content/train_data/"
!unzip "/content/gdrive/MyDrive/GOTCHA/training_07_30.zip" -d /content/train_data &>/dev/null &

Mounted at /content/gdrive/


In [ ]:

from keras import backend as K
from keras.layers import Input, Dense, Flatten, Activation, Dropout, Bidirectional, Permute, multiply ,Layer
from keras.layers.recurrent import SimpleRNN
from keras.callbacks import CSVLogger
from keras.models import Sequential, Model, load_model
from keras.applications.inception_v3 import InceptionV3
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

import os
import time
import numpy as np
import glob
import cv2
from keras.preprocessing.image import img_to_array
from PIL import Image

import datetime

from keras.applications.inception_v3 import preprocess_input
from keras.callbacks import EarlyStopping


In [ ]:
NUM_EPOCHS = 500
VERBOSE = 1
HIDDEN_UNITS = 3

input_data_dir_path = "/content/train_data/training"

In [ ]:
def CNN_feature_extractor(input_data_dir_path, output_feature_dir = ""):
  cnn_model = InceptionV3(include_top = False, weights = 'imagenet' )

  y_samples = []
  x_samples = []


  for f in os.listdir(input_data_dir_path):
    file_path = input_data_dir_path + "/" + f
    if not os.path.exists(file_path + "/extracted_features"):
      os.makedirs(file_path + "/extracted_features")
      output_feature_dir = file_path + "/extracted_features"
    for png_path in glob.glob(file_path + "/father/" + "*.png"):

      info = png_path.split("/")
      print('Extracting features from : ', info[-1])
      
      features = []

      temp = [cv2.imread("/".join(info[0:5] + ["proband"] + [info[-1]])), 
              cv2.imread(png_path) ,
              cv2.imread("/".join(info[0:5] + ["mother"] + [info[-1]]))]

      for image in temp:
        input = cv2.resize(image, (224,224))
        input = img_to_array(input)
        input = np.expand_dims(input, axis = 0)
        input = preprocess_input(input, 'channels_first')
        feature = cnn_model.predict(input).ravel()
        features.append(feature)
      

      unscaled_features = np.array(features)

      np.save(output_feature_dir + "/" + info[-1][:-4] + ".npy", unscaled_features)

      x_samples.append(output_feature_dir + "/" + info[-1][:-4] + ".npy")
      y_samples.append(int((info[-1].split("_")[1]).split(".")[1]) - int((info[-1].split("_")[2]).split(".")[1]))
    return x_samples, y_samples

In [ ]:
def attention_block(inputs, time_steps):
    a = Permute((2, 1))(inputs)
    a = Dense(time_steps, activation='softmax')(a)
    a_probs = Permute((2, 1), name='attention_vec')(a) ## transpose 후 softmax를 빠져나와 다시 한번 더 transpose를 해줌으로서, 중요한 부분에만 가중치가 가도록 하는 역할.
    output_attention_mul = multiply([inputs, a_probs], name='attention_mul') ## input과 attention probability를 곱함.

    return output_attention_mul ## attention module 을 통과한 것을 내보냄.

In [ ]:
def generate_batch(x_samples, y_samples, batch_size):
    num_batches = len(x_samples) // batch_size

    while True:
        for batchIdx in range(0, num_batches):
            start = batchIdx * batch_size
            end = (batchIdx + 1) * batch_size
            x_data = []
            for k in range(start, end):
                x = np.load(x_samples[k])
                indiv = x.shape[0]
                if indiv > 3:
                    x = x[0:3, :]
                    x_data.append(x)
                elif indiv < 3:
                    temp = np.zeros(shape=(3, x.shape[1]))
                    temp[0:indiv, :] = x
                    x_data.append(temp)
                else:
                    x_data.append(x)

            yield np.array(x_data), y_samples[start:end]

In [ ]:
class Classifier(object):
  def __init__(self, model_file = None):
    self.num_input_tokens = None
    self.nb_classes = None
    if model_file is None : self.model = None
    else: self.model = load_model(model_file)



  def cnn_attention_lstm(self):
    inputs = Input(shape=(3, self.num_input_tokens,))
    attention_inputs = attention_block(inputs, 3)
    lstm_out = Bidirectional(SimpleRNN(HIDDEN_UNITS, return_sequences=False))(attention_inputs) ## LSTM --> simple RNN
    x = Dense(512, activation='relu')(lstm_out)
    x = Dropout(0.5)(x)
    x = Dense(self.nb_classes, activation='softmax')(x)
    model = Model([inputs], x)
    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['acc'])
    print(model.summary())
    return model
  
  def fit(self, input_path, output_path, data_set_name = '' , test_size = 0.3, attention = 'cnn_attention_lstm', do_feature_extraction = False):
      batch_size = 128

      cnn_model = InceptionV3(include_top=False, weights='imagenet')

      x_samples, y_samples = CNN_feature_extractor(input_data_dir_path)
      
      labels = dict()
      for y in y_samples:
        if y not in labels:
            labels[y] = len(labels)

      self.num_input_tokens = np.load(x_samples[0]).shape[1]

      self.nb_classes = 64
      y_samples = np_utils.to_categorical(y_samples, self.nb_classes)
        
      config = dict()
      config['labels'] = labels
      config['nb_classes'] = self.nb_classes
      config['num_input_tokens'] = self.num_input_tokens

      np.save(os.path.join(output_path, 'config'), config)


      model = self.cnn_attention_lstm()
      csv_logger = CSVLogger(os.path.join(output_path, 'cnn_attention_lstm.log'), append=True,
                                  separator=';')
      
      Xtrain, Xtest, Ytrain, Ytest = train_test_split(x_samples, y_samples, test_size=test_size,
                                                        random_state=None)
      
      train_gen = generate_batch(Xtrain, Ytrain, batch_size)
      test_gen = generate_batch(Xtest, Ytest, batch_size)

      train_num_batches = len(Xtrain) // batch_size
      test_num_batches = len(Xtest) // batch_size

      history = model.fit_generator(generator=train_gen, steps_per_epoch=train_num_batches,
                              epochs=NUM_EPOCHS,
                              verbose=1, validation_data=test_gen, validation_steps=test_num_batches,
                              callbacks=[csv_logger])

      model_file_path = os.path.join(output_path, attention.replace('cnn', str(datetime.datetime.now())) + '.h5')
      model.save(model_file_path)
      accu = history.history['val_acc'][-1]
      print('cnn-{}, attention-{}: accuracy-{}'.format(str(datetime.datetime.now()), attention, accu))

      return accu

In [ ]:
classifier = Classifier()
classifier.fit(input_path="/content/train_data/training", output_path=".", data_set_name="")

87924736/87910968 [==============================] - 1s 0us/step
Extracting features from :  chr17_s.7359874_vp.7359849.png
Extracting features from :  chr8_s.90605018_vp.90605010.png
Extracting features from :  chr3_s.116169906_vp.116169882.png
Extracting features from :  chr13_s.55044485_vp.55044458.png
Extracting features from :  chr9_s.126685679_vp.126685645.png
Extracting features from :  chr13_s.55044490_vp.55044458.png
Extracting features from :  chr13_s.72874125_vp.72874115.png
Extracting features from :  chr12_s.31094768_vp.31094717.png
Extracting features from :  chr4_s.39471580_vp.39471517.png
Extracting features from :  chr13_s.72874159_vp.72874115.png
Extracting features from :  chr13_s.55044460_vp.55044458.png
Extracting features from :  chr1_s.13317947_vp.13317898.png
Extracting features from :  chr9_s.126685698_vp.126685645.png
Extracting features from :  chr14_s.28801171_vp.28801139.png
Extracting features from :  chr12_s.56354198_vp.56354147.png
Extracting features fr

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/500
5/5 [==============================] - 22s 628ms/step - loss: 4.1701 - acc: 0.0165 - val_loss: 4.1601 - val_acc: 0.0195
Epoch 2/500
5/5 [==============================] - 1s 298ms/step - loss: 4.1572 - acc: 0.0302 - val_loss: 4.1679 - val_acc: 0.0195
Epoch 3/500
5/5 [==============================] - 1s 284ms/step - loss: 4.1368 - acc: 0.0291 - val_loss: 4.1722 - val_acc: 0.0156
Epoch 4/500
5/5 [==============================] - 1s 281ms/step - loss: 4.1153 - acc: 0.0430 - val_loss: 4.1802 - val_acc: 0.0195
Epoch 5/500
5/5 [==============================] - 1s 287ms/step - loss: 4.0921 - acc: 0.0440 - val_loss: 4.1884 - val_acc: 0.0156
Epoch 6/500
5/5 [==============================] - 1s 275ms/step - loss: 4.0854 - acc: 0.0417 - val_loss: 4.1919 - val_acc: 0.0234
Epoch 7/500
5/5 [==============================] - 1s 281ms/step - loss: 4.0731 - acc: 0.0314 - val_loss: 4.2048 - val_acc: 0.0117
Epoch 8/500
5/5 [==============================] - 1s 287ms/step - loss: 4.0472 - 

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


0.01171875